# High Performance Pandas
## `query()`と`eval()`
既にみてきた通り、NumPyやPandasはベクター化した操作を早く行うことを補助する。例えば、2つのarrayの要素を足すときは次の通りである。この操作はループで回すのに比べて、とても速い。

In [1]:
import numpy as np

rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

3.51 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


しかし、この抽象性は複合的な計算を行う時、より非効率となる。例えば、以下のような式を考える。

In [2]:
mask = (x > 0.5) & (y < 0.5)

NumPyはそれぞれの部分式ごとに評価するため、これは大まかに次と同等である。

tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

言い換えれば、それぞれの中間ステップごとに厳密にはメモリーを確保している。もし、`x`と`y`が大きなarrayであったならば、これは大きなメモリー確保と計算のオーバーヘッドを導きかねない。

Numexprライブラリは完全な中間arrayを確保する必要なく、この要素ごとの複合式を計算する力を与える。

このライブラリはNumPyスタイルの式を文字列で受け取ることが出来る。

In [3]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

以下から話す、Pandasの`eval()`や`query()`は、これと概念的に似ており、そしてNumexprパッケージに依存している。

## 効率的な操作のための`pandas.eval()`

In [4]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

In [5]:
%timeit df1 + df2 + df3 + df4

406 ms ± 27.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit pd.eval('df1 + df2 + df3 + df4')

158 ms ± 7.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

True

### `pd.eval()`でサポートされる操作

In [8]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

#### 算術演算

In [9]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### 比較演算

In [10]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### ビット演算

In [11]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [12]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### その他の操作
その他の操作はPandasではサポートしていない。もし必要ならNumexprライブラリを自身を利用できる。

## 列操作に対する`DataFrame.eval()`
Pandasがトップレベルに`pd.eval()`関数を持っているように、`DataFrame`は同じように動く`eval()`メソッドを持っている。`eval()`メソッドの利点は行を名前によって参照することが出来ることである。

In [13]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


`pd.eval()`を利用する場合は以下の様に式を各必要があった。

In [14]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

`DataFrame.eval()`メソッドであれば、より効率的に指揮を書くことが出来る。

In [15]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

### DataFrame.eval()での割り当て

In [16]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


新しい列`D`を作るのに`df.eval()`を利用できる。

In [17]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


同様に、既に存在する列の修正も可能である。

In [18]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### DataFrame.eval()でのローカル変数

In [19]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

`@`は列名でなく変数名であることを示すものであり、これにより2つの名前空間を備える式を効率的に評価できる。列の名前空間と、Pythonオブジェクトの名前空間である。ただし、`@`は`DataFrame.eval()`メソッドのみでサポートされており、`pd.eval()`ではサポートされていないことに注意。なぜなら`pd.eval()`はPythonオブジェクトの名前空間しかアクセス出来ないからである。

## DataFrame.query()メソッド
`DataFrame`は文字列の評価をベースにする`query()`メソッドという他のメソッドもある。以下を考える。

In [20]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

`DataFrame.eval()`は`DataFrame`の列に簡単にアクセスできるようにしたものであるが、このようなフィルター操作には使用できない。代わりに。`query()`メソッドが用意されている。

In [21]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [22]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## これらの関数をいつ使うべきか
これらの関数は計算時間とメモリ使用量から考えて利用すべきである。

計算の中間でメモリ確保が発生しないため、使用するメモリを節約する必要があれば、`eval()`や`query()`を利用すべきである。

計算時間に関しては、CPUのL1、L2キャッシュに乗りきらない程度の計算であれば、`eval()`や`query()`での計算の方が早いかもしれない。

実際のところ、小規模な計算では通常の表現の方が処理が早く、大きなarrayでも計算時間はあまり変わらない。そのため、計算時間の方をとるなら通常の表記、メモリ使用量や構文の分かりやすさを取るなら`eval()`や`query()`の表記とすればよい。